In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from core.pandas_utils import series_start_end_diff
from  transform.processed_tss.ProcessedTimeSeries import TeslaProcessedTimeSeries, ProcessedTimeSeries
from transform.raw_results.config import *
from transform.processed_results.main import make_soh_presentable_per_vehicle
from core.caching_utils import cache_result
from core.stats_utils import mask_out_outliers_by_interquartile_range, force_monotonic_decrease
from transform.raw_results.tesla_results import get_results
from transform.raw_results.mercedes_results import *

In [ ]:
from transform.raw_results.odometer_aggregation import agg_last_odometer
from transform.raw_results.ford_results import get_results as ford_results
from transform.raw_results.tesla_results import get_results as tesla_results
from transform.raw_results.volvo_results import get_results as volvo_results
from transform.raw_results.renault_results import get_results as renault_results
from transform.raw_results.mercedes_results import get_results as mercedes_results

Notebook pour forcer la décroissance des SoH 

## Import data

In [ ]:
df_ford = ford_results(force_update=False).dropna(subset='soh')
df_mercedes = mercedes_results(force_update=False).dropna(subset='soh')
df_renault = renault_results(force_update=False).dropna(subset='soh')
df_tesla = tesla_results(force_update=False).dropna(subset='soh')
df_volvo = volvo_results(force_update=False).dropna(subset='soh')

In [ ]:
df_ford.shape, df_mercedes.shape, df_renault.shape, df_tesla.shape, df_volvo.shape

## Appliquer un decroissance avec un calcul de windows fonction 

Remplace ce qui est fait aujourd'hui avec la fonction `force_monotonic_decrease`

In [ ]:
def generate_decreasing_series(df, window_size=3, max_drop=0.003):
    """
    Génère une série strictement décroissante à partir d'une liste de valeurs en :
      - Calculant d'abord une moyenne mobile (pour lisser les valeurs),
      - En choisissant un point de départ proche du maximum (pour refléter les meilleures valeurs),
      - Puis en s'assurant qu'entre deux points consécutifs, la diminution ne dépasse pas max_drop,
        et que la série ne stagne pas sur plus de 2 points (on garantit ici une décroissance stricte).
    """
    # Calcul de la moyenne mobile
    smoothed = df['soh'].rolling(window=window_size, min_periods=2).mean().values
    # récupère l'odomètre pour la dcroissance forcé.
    odometer = df['odometer'].ffill().copy().values
    n = len(smoothed)
    
    # Pour le démarrage, on part du maximum de la série lissée / pas forcéement optimal
    output = [max(smoothed[1:])]
    for i in range(1, n):
        candidate = smoothed[i]
        prev = output[-1]
        
        # Si  candidate >= à la précédente, on la force à être légèrement plus basse en se basant sur l'odomètre
        if candidate >= prev:
            epsilon = (odometer[i] - odometer[i-1]) * 1e-7
            candidate = prev - epsilon

        
        # Vérifier que le drop n'est pas trop important
        drop = prev - candidate
        if drop > max_drop:
            candidate = prev - max_drop
        output.append(candidate)
        
    return output



## Tesla 

In [ ]:
df_vin = df_tesla[df_tesla['vin']=="XP7YGCEK2RB289163"].copy()
mask = mask_out_outliers_by_interquartile_range(df_vin["soh"])
df_vin_test = df_vin[mask].copy()

`actual method`

In [ ]:
df_vin_test["soh_actual_method"] = force_monotonic_decrease(df_vin_test["soh"]).values

`new method`

In [ ]:
df_vin_test['rolling_soh'] = generate_decreasing_series(df_vin_test)

Result

In [ ]:
df_vin_test[["odometer", "soh", "soh_actual_method", "rolling_soh", "date"]]


Plot 

In [ ]:
fig = go.Figure() 
fig.add_trace(go.Scatter(x=df_vin_test['odometer'], y=df_vin_test['soh'], mode="markers", opacity=1, name='get result'), )
fig.add_trace(go.Scatter(x=df_vin_test['odometer'], y=df_vin_test['rolling_soh'], mode="markers", opacity=1, name='rolling'), )
fig.add_trace(go.Scatter(x=df_vin_test['odometer'], y=df_vin_test['soh_actual_method'], mode="markers", opacity=1, name='actual method'), )


## ford 

In [ ]:

# ça prend beaucoup de temps chosir un autre vin
df_vin = df_ford[df_ford['vin']=="WF0TK1EM5MMA31333"].copy()
mask = mask_out_outliers_by_interquartile_range(df_vin["soh"])
df_vin_test = df_vin[mask].copy()

In [ ]:
df_vin_test.shape

`actual method`

In [ ]:
df_vin_test["soh_actual_method"] = force_monotonic_decrease(df_vin_test["soh"]).values

`new method`

In [ ]:
df_vin_test['rolling_soh'] = generate_decreasing_series(df_vin_test)

Result

In [ ]:
df_vin_test[["odometer", "soh", "soh_actual_method", "rolling_soh", "date"]]


Plot 

In [ ]:
fig = go.Figure() 
fig.add_trace(go.Scatter(x=df_vin_test['odometer'], y=df_vin_test['soh'], mode="markers", opacity=1, name='get result'), )
fig.add_trace(go.Scatter(x=df_vin_test['odometer'], y=df_vin_test['rolling_soh'], mode="markers", opacity=1, name='rolling'), )
fig.add_trace(go.Scatter(x=df_vin_test['odometer'], y=df_vin_test['soh_actual_method'], mode="markers", opacity=1, name='actual method'), )


## Renault

In [ ]:
df_vin = df_renault[df_renault['vin']=="VF1AG000664474289"].copy()
mask = mask_out_outliers_by_interquartile_range(df_vin["soh"])
df_vin_test = df_vin[mask].copy()

`actual method`

In [ ]:
df_vin_test["soh_actual_method"] = force_monotonic_decrease(df_vin_test["soh"]).values

`new method`

In [ ]:
df_vin_test['rolling_soh'] = generate_decreasing_series(df_vin_test)

Result

In [ ]:
df_vin_test[["odometer", "soh", "soh_actual_method", "rolling_soh", "date"]]


Plot 

In [ ]:
fig = go.Figure() 
fig.add_trace(go.Scatter(x=df_vin_test['odometer'], y=df_vin_test['soh'], mode="markers", opacity=1, name='get result'), )
fig.add_trace(go.Scatter(x=df_vin_test['odometer'], y=df_vin_test['rolling_soh'], mode="markers", opacity=1, name='rolling'), )
fig.add_trace(go.Scatter(x=df_vin_test['odometer'], y=df_vin_test['soh_actual_method'], mode="markers", opacity=1, name='actual method'), )
